In [1]:
%matplotlib inline
import tensorflow as tf
from vahun.Text import Text
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
from vahun.variational_autoencoder import Variational_autoencoder
from vahun.genetic import evolution
from vahun.genetic import experiment
from vahun.tools import show_performance

encode=180

popsize=1


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

corpus=Text(corpus_path='/mnt/store/velkey/mnsz2/filt.200k.maxlen20',size=100000)

logger=explog(encoder_type="hunfiltered_lowered_uniq_"+str(encode),
              encoding_dim=encode,feature_len=10,
              lang="Hun",unique_words=len(set(corpus.wordlist)),
              name="autoencoder_lowered_"+str(encode),population_size=popsize,
              words=len(corpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

exp=experiment(encoded_width=10,layermax=10,layermin=2,maxw=10,minw=3,out_dim=corpus.abc*10)

exp.weights=[encode, len(corpus.abc)*10]
exp.len=len(exp.weights)

In [4]:
encoder=Variational_autoencoder(logger=logger,tf_session=sess,
                         inputdim=960,
                         encoding_size=400,corpus=corpus,
                         optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                         nonlinear=tf.sigmoid)
encoder.train(corpus.x_train,corpus.x_valid,corpus.x_test,512,80)

ValueError: cannot reshape array of size 491520 into shape (512,0)

In [6]:
stds=show_performance(encoder,corpus.x_test,corpus,10,printer=True,inputfsize=48)

szálanként 	 szálanként 	 0
 aratáskor 	  aratáskor 	 0
      domb 	       dobt 	 2
 találjunk 	  találjunk 	 0
többségére 	 tábbségére 	 1
  láthatta 	   láthatta 	 0
     birom 	      birom 	 0
   kihívja 	    kihívja 	 0
  kamatozó 	   kamatozó 	 0
üzenetemet 	 szenetemet 	 1

Accuracy on data:  94.37092391304348 %
average Levenshtein distance:  0.566168478261


In [8]:
stds=show_performance(encoder,["e","eh","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],corpus,printer=True,inputfsize=48)

         e 	         ve 	 1
        eh 	         et 	 1
     kutya 	      kutya 	 0
     aytuk 	      aytuk 	 0
    macska 	     macska 	 0
 árvíztűrő 	  árviztörő 	 2
      fúró 	       híró 	 2
     kacsa 	      kapsa 	 1
         a 	         ta 	 1
        és 	         és 	 0

Accuracy on data:  92.0 %
average Levenshtein distance:  0.8


In [9]:
def decode_critical(enc=180):
    for POS in range(enc):
        encoded=np.ones(enc)
        encoded[POS]=0
        a=encoder.decode([encoded])[0].reshape([10,36])

        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)